In [1]:
from tickers import Ticker, scrape_tickers, query_list, recent_IPO_list, upcoming_IPO_list
import pandas as pd

# Scrape to get updated tickers and company names for stocks and ETFs
scrape_tickers()

import_lists = [query_list, recent_IPO_list, upcoming_IPO_list]

tickers = []

# Build raw query list with ticker abbrevation and name of company
for i in import_lists:
    for item in i:
        tickers.append(item)

In [2]:
# Load in data for testing - will be replace with dataframes when in production
submissions_df = pd.read_csv('submissions.csv')
comments_df = pd.read_csv('comments.csv')

In [3]:
# convert sentences to array of words
submissions_df['Body_Words'] = submissions_df.Body.str.replace("'", "").str.strip().str.split('[\W_]+')
submissions_df['Title_Words'] = submissions_df.Title.str.replace("'", "").str.strip().str.split('[\W_]+')

In [4]:
common_words = ['Corp', 'Corporation', 'Ltd', 'Acquisition', 'Pharmaceuticals', 'Holding', 'Group']
new_name = ""
ticker_names = []
for ticker in tickers:
    new_name = ""
    company_words = ticker.name.split()
    for word in company_words:
        if word in common_words:
            continue
        else:
            if new_name == "":
                new_name = word
            else:
                new_name = new_name + " " + word
    ticker.name = new_name

In [5]:
# Tag all tickers found in array of words for each post
submissions_df["Tickers"] = ""
for index, row in submissions_df.iterrows():
    tickers_found = []
    body_list = row[12]
    title_list = row[13]
    body_set = set(body_list)
    title_set = set(title_list)
    for ticker in tickers:
        if ticker.abbrev == 'A' or ticker.abbrev == 'I':
            if (ticker.name in body_set) or (ticker.name in title_set):
                tickers_found.append(ticker.abbrev)
        else:
            if (ticker.abbrev in body_set) or (ticker.abbrev in title_set):
                if ticker.abbrev not in tickers_found:
                    tickers_found.append(ticker.abbrev)
            elif (ticker.name in body_set) or (ticker.name in title_set):
                if ticker.abbrev not in tickers_found:
                    tickers_found.append(ticker.abbrev)
    if len(tickers_found) == 0:
        submissions_df.at[index, "Tickers"] = ""
    else:
        submissions_df.at[index, "Tickers"]= tickers_found
        print(f"{index}: {tickers_found}")

1: ['ADS', 'BCS', 'DRI', 'HON', 'JP', 'MS', 'NDAQ', 'NIO', 'ORLY', 'REGN', 'RH', 'SGFY', 'TLS', 'U', 'DIA', 'QQQ', 'SPY']
2: ['DG', 'E', 'GOOGL', 'HD', 'SPGI', 'TMO', 'TXN', 'UNP', 'V']
3: ['PS']
4: ['AM', 'AMAT', 'BB', 'BNTX', 'CHWY', 'CIO', 'ENPH', 'ET', 'ETSY', 'FCX', 'FOR', 'GILD', 'KMX', 'KNDI', 'L', 'LB', 'LPL', 'LULU', 'MKC', 'MKC.V', 'MTD', 'MU', 'NDAQ', 'NEXT', 'NWG', 'OPEN', 'PM', 'SE', 'TREE', 'TSLA', 'TWOA', 'U', 'VIAC', 'VIACA', 'WBA']
6: ['FANG', 'PLTR', 'PS']
7: ['MX']
10: ['USD']
11: ['DD']
12: ['VIAC']
13: ['CEO', 'FRO', 'NAT', 'STNG', 'TK', 'TNK']
14: ['ARKK']
15: ['F', 'NIO', 'USD']
16: ['DD']
17: ['AMT', 'CCI', 'CTRE', 'EPRT', 'EQIX', 'INVH', 'NSA', 'PLD', 'PSB', 'REXR', 'REIT']
19: ['TSLA', 'VWAGY']
20: ['TWTR']
22: ['GME']
23: ['H', 'IJR', 'IWC', 'IWM', 'SDD']
28: ['CP']
29: ['TD']
30: ['NDAQ', 'TA', 'TWOA']
31: ['X', 'Y']
32: ['TMAC']
34: ['MSFT']
36: ['GME', 'TWOA', 'ARKK', 'LIT', 'MSOS', 'YOLO']
38: ['CEO', 'DHT', 'DSX', 'FRO', 'NAT', 'SFL', 'TNP', 'TWOA']
39: 

In [6]:
# Remove submissions without any mention of tickers - unable to interpret relevancy of sentiment
submissions_df = submissions_df.loc[submissions_df['Tickers'] != ""]

In [7]:
import nltk
#nltk.download('punkt')

In [8]:
# Using Vader Sentiment Analysis on Submissions
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from pprint import pprint
#nltk.download('vader_lexicon')
sia = SIA()
results = []
submissions_df['negative_score'] = ""
submissions_df['neutral_score'] = ""
submissions_df['positive_score'] = ""
submissions_df['compound_score'] = ""
for index, row in submissions_df.iterrows():
    title = row[2]
    body = row[4]
    eval_text = title + " " + body
    body_words = row[12]
    title_words = row[13]
   # ticker_abbrevs = row[14]
    pol_score= sia.polarity_scores(eval_text)
    submissions_df.at[index, 'negative_score'] = pol_score['neg']
    submissions_df.at[index, 'neutral_score'] = pol_score['neu']
    submissions_df.at[index, 'positive_score'] = pol_score['pos']
    submissions_df.at[index, 'compound_score'] = pol_score['compound']

submissions_df.head()

,Post_Type,Submission_ID,Title,Author,Body,Flair,Distinguished,Num_Comments,Post_ID,URL,Score,Upvote_Ratio,Created_Date_UTC,Body_Words,Title_Words,Tickers,negative_score,neutral_score,positive_score,compound_score
1,Submission,me2d54,"Here is a Market Recap for today Friday, March...",psychotrader00,"\n\n**PsychoMarket Recap - Friday, March 26, ...",NaN,NaN,21,t3_me2d54,https://www.reddit.com/r/stocks/comments/me2d5...,72,0.92,[1616804555.0],"[, PsychoMarket, Recap, Friday, March, 26, 202...","[Here, is, a, Market, Recap, for, today, Frida...","[ADS, BCS, DRI, HON, JP, MS, NDAQ, NIO, ORLY, ...",0.045,0.848,0.107,0.9937
2,Submission,me8kb3,5 Considerations When Picking Stocks for Long-...,MinnesotaPower,"Like many of you, I started actively investing...",Advice,NaN,18,t3_me8kb3,https://www.reddit.com/r/stocks/comments/me8kb...,81,0.87,[1616827758.0],"[Like, many, of, you, I, started, actively, in...","[5, Considerations, When, Picking, Stocks, for...","[DG, E, GOOGL, HD, SPGI, TMO, TXN, UNP, V]",0.042,0.836,0.122,0.9842
3,Submission,me1wx4,I believe two contradictory things about stock...,NotLegallyBinding,"First, I'm convinced that all the relevant, ac...",NaN,NaN,97,t3_me1wx4,https://www.reddit.com/r/stocks/comments/me1wx...,61,0.82,[1616803173.0],"[First, Im, convinced, that, all, the, relevan...","[I, believe, two, contradictory, things, about...",[PS],0.142,0.778,0.081,-0.7559
4,Submission,med6uw,Wall Street Week Ahead for the trading week be...,bigbear0083,Good Saturday morning to all of you here on r/...,NaN,NaN,19,t3_med6uw,https://www.reddit.com/r/stocks/comments/med6u...,97,0.96,[1616849490.0],"[Good, Saturday, morning, to, all, of, you, he...","[Wall, Street, Week, Ahead, for, the, trading,...","[AM, AMAT, BB, BNTX, CHWY, CIO, ENPH, ET, ETSY...",0.031,0.871,0.098,0.9999
6,Submission,mdwibp,PLTR - bearish discussion to better understand...,wsbloverrrrrr,"(Please excuse my username, it's the only acco...",Company Discussion,NaN,58,t3_mdwibp,https://www.reddit.com/r/stocks/comments/mdwib...,43,0.79,[1616787276.0],"[, Please, excuse, my, username, its, the, onl...","[PLTR, bearish, discussion, to, better, unders...","[FANG, PLTR, PS]",0.066,0.746,0.188,0.9981


In [9]:
submissions_df['sentiment_label'] = 0
submissions_df.loc[submissions_df['compound_score'] > 0.2, 'sentiment_label'] = 1
submissions_df.loc[submissions_df['compound_score'] < -0.2, 'sentiment_label'] = -1

In [10]:
submissions_df['sentiment_weight'] = submissions_df['Score'] * submissions_df['Upvote_Ratio']

In [11]:
comments_df['Body_Words'] = comments_df.Body.str.replace("'", "").str.strip().str.split('[\W_]+')

In [12]:
comments_df['Tickers'] = ""
for index_c, row_c in comments_df.iterrows():
    for index_s, row_s in submissions_df.iterrows():
        if row_c[1] == row_s[1]:
             comments_df.at[index_c, 'Tickers'] = row_s[14]

In [13]:
comments_df = comments_df.loc[comments_df["Tickers"] != ""]

In [14]:
comments_df['negative_score'] = ""
comments_df['neutral_score'] = ""
comments_df['positive_score'] = ""
comments_df['compound_score'] = ""
for index, row in comments_df.iterrows():
    body = row[5]
    pol_score= sia.polarity_scores(body)
    comments_df.at[index, 'negative_score'] = pol_score['neg']
    comments_df.at[index, 'neutral_score'] = pol_score['neu']
    comments_df.at[index, 'positive_score'] = pol_score['pos']
    comments_df.at[index, 'compound_score'] = pol_score['compound']

comments_df.head()

,Post_Type,Submission_ID,Comment_ID,Parent_ID,Author,Body,Flair,Distinguished,Is_Author,Score,Created_Date_UTC,Body_Words,Tickers,negative_score,neutral_score,positive_score,compound_score
187,Comment,me2d54,gsd1r54,t3_me2d54,pman6,['ok guys....\n\nwhat the fuck happened during...,NaN,NaN,False,35,[1616805654.0],"[, ok, guys, n, nwhat, the, fuck, happened, du...","[Here, is, a, Market, Recap, for, today, Frida...",0.224,0.776,0,-0.7236
188,Comment,me2d54,gsdbbya,t3_me2d54,NeelAsman,"['Shout out for yesterday’s post, you nailed S...",NaN,NaN,False,5,[1616809931.0],"[, Shout, out, for, yesterday, s, post, you, n...","[Here, is, a, Market, Recap, for, today, Frida...",0,1,0,0
189,Comment,me2d54,gsd2vy0,t3_me2d54,thatsjetfuel,"[""I'm not seeing how TSM is a bad play at this...",NaN,NaN,False,7,[1616806145.0],"[, Im, not, seeing, how, TSM, is, a, bad, play...","[Here, is, a, Market, Recap, for, today, Frida...",0.147,0.767,0.087,-0.5423
190,Comment,me2d54,gsd61xo,t1_gsd1r54,hitmon_gg,['God wanted me to have a better weekend?'],NaN,NaN,False,23,[1616807556.0],"[, God, wanted, me, to, have, a, better, weeke...","[Here, is, a, Market, Recap, for, today, Frida...",0,0.674,0.326,0.4404
191,Comment,me2d54,gsddp75,t1_gsd1r54,bothodler,"[""I don't want to get super conspiracy-ie but ...",NaN,NaN,False,10,[1616811018.0],"[, I, dont, want, to, get, super, conspiracy, ...","[Here, is, a, Market, Recap, for, today, Frida...",0.299,0.598,0.102,-0.9497


In [15]:
comments_df['sentiment_label'] = 0
comments_df.loc[comments_df['compound_score'] > 0.2, 'sentiment_label'] = 1
comments_df.loc[comments_df['compound_score'] < -0.2, 'sentiment_label'] = -1
comments_df

,Post_Type,Submission_ID,Comment_ID,Parent_ID,Author,Body,Flair,Distinguished,Is_Author,Score,Created_Date_UTC,Body_Words,Tickers,negative_score,neutral_score,positive_score,compound_score,sentiment_label
187,Comment,me2d54,gsd1r54,t3_me2d54,pman6,['ok guys....\n\nwhat the fuck happened during...,NaN,NaN,False,35,[1616805654.0],"[, ok, guys, n, nwhat, the, fuck, happened, du...","[Here, is, a, Market, Recap, for, today, Frida...",0.224,0.776,0,-0.7236,-1
188,Comment,me2d54,gsdbbya,t3_me2d54,NeelAsman,"['Shout out for yesterday’s post, you nailed S...",NaN,NaN,False,5,[1616809931.0],"[, Shout, out, for, yesterday, s, post, you, n...","[Here, is, a, Market, Recap, for, today, Frida...",0,1,0,0,0
189,Comment,me2d54,gsd2vy0,t3_me2d54,thatsjetfuel,"[""I'm not seeing how TSM is a bad play at this...",NaN,NaN,False,7,[1616806145.0],"[, Im, not, seeing, how, TSM, is, a, bad, play...","[Here, is, a, Market, Recap, for, today, Frida...",0.147,0.767,0.087,-0.5423,-1
190,Comment,me2d54,gsd61xo,t1_gsd1r54,hitmon_gg,['God wanted me to have a better weekend?'],NaN,NaN,False,23,[1616807556.0],"[, God, wanted, me, to, have, a, better, weeke...","[Here, is, a, Market, Recap, for, today, Frida...",0,0.674,0.326,0.4404,1
191,Comment,me2d54,gsddp75,t1_gsd1r54,bothodler,"[""I don't want to get super conspiracy-ie but ...",NaN,NaN,False,10,[1616811018.0],"[, I, dont, want, to, get, super, conspiracy, ...","[Here, is, a, Market, Recap, for, today, Frida...",0.299,0.598,0.102,-0.9497,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2111,Comment,me11cn,gsdhaar,t1_gsdf2zh,merlinsbeers,"[""He isn't self-made. He comes from a wealthy ...",NaN,NaN,False,2,[1616812668.0],"[, He, isnt, self, made, He, comes, from, a, w...","[Elons, pumps, lost, its, effect, Totally, uns...",0,0.846,0.154,0.6249,1
2112,Comment,me11cn,gsge5va,t1_gsdf2zh,harrison_wintergreen,['> That dude is one of the few self made bill...,NaN,NaN,False,1,[1616856792.0],"[, That, dude, is, one, of, the, few, self, ma...","[Elons, pumps, lost, its, effect, Totally, uns...",0,1,0,0,0
2113,Comment,me11cn,gsd6jo3,t1_gscuc3i,Ironleg01,"['Oh wow, news outlet reporting on thing that ...",NaN,NaN,False,4,[1616807785.0],"[, Oh, wow, news, outlet, reporting, on, thing...","[Elons, pumps, lost, its, effect, Totally, uns...",0.296,0.51,0.194,-0.25,-1
2114,Comment,me11cn,gsd0mbr,t1_gsczs3d,Tacoman404,"['Huh, Elon Musk is actually a vole?']",NaN,NaN,False,2,[1616805158.0],"[, Huh, Elon, Musk, is, actually, a, vole, ]","[Elons, pumps, lost, its, effect, Totally, uns...",0,1,0,0,0


In [16]:
sentiment_average_df = comments_df[['Submission_ID','sentiment_label']]

In [17]:
sentiment_average_df['sentiment_weight'] = comments_df['Score']
sentiment_average_df

/Users/madisonleopold/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Submission_ID,sentiment_label,sentiment_weight
187,me2d54,-1,35
188,me2d54,0,5
189,me2d54,-1,7
190,me2d54,1,23
191,me2d54,-1,10
...,...,...,...
2111,me11cn,1,2
2112,me11cn,0,1
2113,me11cn,-1,4
2114,me11cn,0,2


In [18]:
sentiment_average_df['sentiment_adjusted'] = sentiment_average_df['sentiment_label'] * sentiment_average_df['sentiment_weight']

/Users/madisonleopold/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [19]:
sentiment_average = sentiment_average_df[['Submission_ID','sentiment_adjusted']].groupby('Submission_ID').mean()

In [20]:
sentiment_average['sentiment_output'] = 0
sentiment_average.loc[sentiment_average['sentiment_adjusted'] > 0.2, 'sentiment_output'] = 1
sentiment_average.loc[sentiment_average['sentiment_adjusted'] < -0.2, 'sentiment_output'] = -1
sentiment_average.reset_index(inplace=True)
#sentiment_average = sentiment_average.set_index('Submission_ID')

In [21]:
sentiment_average

,Submission_ID,sentiment_adjusted,sentiment_output
0,mdqmi5,-2.333333,-1
1,mdr73u,0.461538,1
2,mdrafp,-0.200000,0
3,mdrf39,-0.200000,0
4,mdrs63,1.571429,1
5,mds0i6,0.500000,1
6,mds4bo,0.400000,1
7,mdsfjo,0.888889,1
8,mdssz4,0.083333,0
9,mdt1eq,1.068966,1


In [22]:
#combined_df = submissions_df.join(sentiment_average)
combined_df = pd.merge(submissions_df, sentiment_average, how='left', on='Submission_ID',
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

In [23]:
combined_df.rename(columns = {'sentiment_adjusted':'comment_sentiment'}, inplace = True)

In [24]:
combined_df['submission_sentiment'] = combined_df['sentiment_label'] * combined_df['sentiment_weight']

In [25]:
combined_df.head()

,Post_Type,Submission_ID,Title,Author,Body,Flair,Distinguished,Num_Comments,Post_ID,URL,...,Tickers,negative_score,neutral_score,positive_score,compound_score,sentiment_label,sentiment_weight,comment_sentiment,sentiment_output,submission_sentiment
0,Submission,mdqmi5,$GSX - Options activity,SaveAmerica21,$GSX - question. What is up with all the Chine...,NaN,NaN,3,t3_mdqmi5,https://www.reddit.com/r/stocks/comments/mdqmi...,...,"[AMBO, CLEU, EEIQ, FEDU, WAFU]",0,0.957,0.043,0.34,1,3.00,-2.333333,-1.0,3.00
1,Submission,mdr73u,Short term tanker stocks for Suez Canal blocka...,Rock_it_Scientist,"Ok, so we all know that some quarter mile lon...",Industry Discussion,NaN,14,t3_mdr73u,https://www.reddit.com/r/stocks/comments/mdr73...,...,"[CEO, DHT, DSX, FRO, NAT, SFL, TNP, TWOA]",0.08,0.869,0.051,-0.9834,-1,5.39,0.461538,1.0,-5.39
2,Submission,mdrafp,XPO Logistics Inc. (XPO) Soars 2.43% on March 25,thinkB4WeSpeak,XPO Logistics Inc. (XPO) had a good day on the...,Company News,NaN,5,t3_mdrafp,https://www.reddit.com/r/stocks/comments/mdraf...,...,"[CEO, TWOA, XPO, LTL]",0.026,0.908,0.066,0.9169,1,0.00,-0.200000,0.0,0.00
3,Submission,mdrf39,"Rookie Mistake, left with a question.",Cindermidnight2,So I misunderstood RobinHood’s day trading war...,Advice,NaN,57,t3_mdrf39,https://www.reddit.com/r/stocks/comments/mdrf3...,...,[TD],0.081,0.769,0.15,0.9669,1,8.47,-0.200000,0.0,8.47
4,Submission,mdrs63,Dollar cost average,noeljvt,Is there a way to calculate dollar cost averag...,NaN,NaN,14,t3_mdrs63,https://www.reddit.com/r/stocks/comments/mdrs6...,...,[TWOA],0,0.905,0.095,0.6046,1,0.00,1.571429,1.0,0.00


In [26]:
combined_df['combined_sentiment'] = combined_df['comment_sentiment'] + combined_df['submission_sentiment']

In [27]:
combined_df['final_sentiment_label'] = 0
combined_df.loc[combined_df['combined_sentiment'] > 0.2, 'final_sentiment_label'] = 1
combined_df.loc[combined_df['combined_sentiment'] < -0.2, 'final_sentiment_label'] = -1

In [28]:
ticker_sentiment = pd.DataFrame()

In [29]:
ticker_sentiment["Ticker"] = ""
ticker_sentiment["Total_Count"] = 0
ticker_sentiment["Total_Sentiment"] = 0

In [30]:
index = 0
for ticker in tickers:
    ticker_sentiment.at[index, "Ticker"] = ticker.abbrev
    ticker_sentiment.at[index, "Total_Count"] = 0
    ticker_sentiment.at[index, "Total_Sentiment"] = 0
    index += 1

In [31]:
ticker_sentiment = ticker_sentiment.drop_duplicates(subset=['Ticker'])

In [32]:
ticker_sentiment

,Ticker,Total_Count,Total_Sentiment
0,A,0.0,0.0
1,AA,0.0,0.0
2,AAC,0.0,0.0
3,AACE,0.0,0.0
4,AACG,0.0,0.0
...,...,...,...
8911,YYY,0.0,0.0
8912,ZIG,0.0,0.0
8913,ZJPN,0.0,0.0
8914,ZROZ,0.0,0.0


In [33]:
# Get running count of tickers mentioned (# of times mentioned) and with average sentiment
for index, row in ticker_sentiment.iterrows():
    ticker_abbrev = row[0]
    total_count = row[1]
    total_sentiment = row[2]
    for i, r in combined_df.iterrows():
        # Get column with tickers found
        ticker_set = set(r[15])
        post_sentiment = r[26]
        if ticker_abbrev in ticker_set:
            total_count += 1
            total_sentiment += post_sentiment
    ticker_sentiment.at[index, "Total_Count"] = total_count
    ticker_sentiment.at[index, "Total_Sentiment"] = total_sentiment

In [34]:
final_sentiment_results = ticker_sentiment.loc[ticker_sentiment['Total_Count'] != 0]

In [35]:
final_sentiment_results = final_sentiment_results.reset_index(drop=True)

In [36]:
final_sentiment_results

,Ticker,Total_Count,Total_Sentiment
0,ADS,1.0,1.0
1,AM,1.0,1.0
2,AMAT,1.0,1.0
3,AMBO,1.0,1.0
4,AMC,2.0,2.0
...,...,...,...
150,VIS,1.0,1.0
151,VOO,1.0,1.0
152,VTI,1.0,1.0
153,VUG,1.0,1.0


In [37]:
final_sentiment_results["Average_Sentiment"] = final_sentiment_results["Total_Count"] / final_sentiment_results["Total_Sentiment"]

In [38]:
final_sentiment_results

,Ticker,Total_Count,Total_Sentiment,Average_Sentiment
0,ADS,1.0,1.0,1.0
1,AM,1.0,1.0,1.0
2,AMAT,1.0,1.0,1.0
3,AMBO,1.0,1.0,1.0
4,AMC,2.0,2.0,1.0
...,...,...,...,...
150,VIS,1.0,1.0,1.0
151,VOO,1.0,1.0,1.0
152,VTI,1.0,1.0,1.0
153,VUG,1.0,1.0,1.0


In [39]:
# Add "today's price change" to each ticker found - total count, average sentiment, today's price change
# Include graph for visuals
import yfinance as yf
from datetime import date

In [40]:
todays_date = date.today()
todays_date

datetime.date(2021, 5, 22)

In [41]:
final_sentiment_results["Price_Change"] = ""
final_sentiment_results["Date"] = ""
for index, row in final_sentiment_results.iterrows():
    try:
        df = yf.download(row[0], start=todays_date)
        date = df.index[0]
        close_price = df["Close"].values[0]
        open_price = df["Open"].values[0]
        percentage_change = round((((close_price - open_price) / open_price) * 100), 2)
        final_sentiment_results.at[index, "Price_Change"] = f'{percentage_change}%'
        final_sentiment_results.at[index, "Date"] = date
    except:
        print("Data not found")
        final_sentiment_results.at[index, "Price_Change"] = "Ticker unavailable"
        final_sentiment_results.at[index, "Date"] = date

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- ARK: No data found for this date range, symbol may be delisted
Data not found
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[**

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [42]:
final_sentiment_results

,Ticker,Total_Count,Total_Sentiment,Average_Sentiment,Price_Change,Date
0,ADS,1.0,1.0,1.0,-0.08%,2021-05-21 00:00:00
1,AM,1.0,1.0,1.0,-0.99%,2021-05-21 00:00:00
2,AMAT,1.0,1.0,1.0,-0.92%,2021-05-21 00:00:00
3,AMBO,1.0,1.0,1.0,3.17%,2021-05-21 00:00:00
4,AMC,2.0,2.0,1.0,-4.2%,2021-05-21 00:00:00
...,...,...,...,...,...,...
150,VIS,1.0,1.0,1.0,-0.04%,2021-05-21 00:00:00
151,VOO,1.0,1.0,1.0,-0.46%,2021-05-21 00:00:00
152,VTI,1.0,1.0,1.0,-0.51%,2021-05-21 00:00:00
153,VUG,1.0,1.0,1.0,-0.89%,2021-05-21 00:00:00


# Test database connection and process

In [43]:
# Import SQL Alchemy
from sqlalchemy import create_engine

# Import datetime
from datetime import datetime

# Import and establish Base for which classes will be constructed 
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float

In [44]:
# Create the Ticker Sentiment class
class Ticker_Sentiment(Base):
    __tablename__ = 'ticker_sentiment'
    id = Column(Integer, primary_key=True)
    ticker = Column(String(255))
    date = Column(String(255))
    count = Column(Integer)
    sentiment = Column(Integer)
    percent_change = Column(Float)

In [45]:
# Create a connection to a SQLite database
engine = create_engine('sqlite:///reddit_sentiment.db')

In [46]:
# Create the ticker_sentiment table within the database
Base.metadata.create_all(engine)

In [47]:
# Push the objects made and query the server
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [48]:
sql_df = final_sentiment_results[["Ticker", "Date", "Total_Count", "Average_Sentiment", "Price_Change"]]

In [49]:
sql_df = sql_df.rename(columns={"Ticker":"ticker", "Date":"date", "Total_Count":"count", "Average_Sentiment":"sentiment", "Price_Change":"percent_change"})

In [50]:
sql_df

,ticker,date,count,sentiment,percent_change
0,ADS,2021-05-21 00:00:00,1.0,1.0,-0.08%
1,AM,2021-05-21 00:00:00,1.0,1.0,-0.99%
2,AMAT,2021-05-21 00:00:00,1.0,1.0,-0.92%
3,AMBO,2021-05-21 00:00:00,1.0,1.0,3.17%
4,AMC,2021-05-21 00:00:00,2.0,1.0,-4.2%
...,...,...,...,...,...
150,VIS,2021-05-21 00:00:00,1.0,1.0,-0.04%
151,VOO,2021-05-21 00:00:00,1.0,1.0,-0.46%
152,VTI,2021-05-21 00:00:00,1.0,1.0,-0.51%
153,VUG,2021-05-21 00:00:00,1.0,1.0,-0.89%


In [52]:
sql_df.to_sql('ticker_sentiment', con=engine, if_exists='append', index=False)